# Assumptions:

*   The minimum distance between a start note and destination node of a customer is 4
*   The fleet of cars start of from the same position each day (node 0 for simplicity)
*   A pick up/drop off takes one tick of the simulation clock. In other words, a car can pick up/drop off multiple people at one node but can't visit and pick up/drop off at another node in the span of one tick. 



In [ ]:
import networkx as nx

In [ ]:
# Generate a random graph in which our cars will live.
# "The graph that you will use consists of 200 nodes, with an average connectivity of 2."
from itertools import combinations
from random import sample, choice
from math import ceil
from typing import Tuple, List, Dict


def generate_road_network(num_nodes:int, avg_connectivity:int, 
                          proportion_of_edges_to_redo:float = 1/4) -> nx.Graph:
  """
  Generate a road network of `num_nodes` with average degree `avg_connectivity`, but is not completely regular.
  :param num_nodes: int representing number of nodes in a network
  :param avg_connectivity: int showing the average degree of nodes in the network
  :param proportion_of_edges_to_redo: float representing the proportion of edges randomly redo, so the graph isn't perfectly regular.
  :returns: a NetworkX Graph object representing the network
  """
  roads = nx.generators.random_graphs.random_regular_graph(avg_connectivity, 
                                                          num_nodes)

  # This, however, is just loop(s), since it's a regular graph. Two solutions
  # include either take this graph and add/remove equal number of edges. If we do
  # this, we must also check for separate components in the graph

  random_edges = sample(roads.edges, ceil(proportion_of_edges_to_redo*len(roads.edges)))

  for edge in random_edges:
    # make sure we don't disconnect any nodes
    if roads.degree[edge[0]] > 1 and roads.degree[edge[1]] > 1:
      new_edge = sample(roads.nodes, 2)
      # Make sure we don't duplicate any nodes
      if new_edge not in roads.edges:
        roads.remove_edge(*edge)
        roads.add_edge(*new_edge)


  # Make sure it's only got one component
  # This may repeat, since there is no check that the edge replacements below do 
  # not separate the graph.
  while not nx.is_connected(roads):
    components = nx.connected_components(roads)
    random_path = []
    # Make a path that connects every component
    for component in components:
      random_path.append(sample(component,1)[0])
    # Remove random edges
    roads.remove_edges_from(sample(roads.edges, len(random_path)-1))
    # Add new path
    nx.add_path(roads, random_path)
  return roads

test_roads = generate_road_network(200, 3)


In [ ]:
# Draw Roads
nx.draw(test_roads)

In [ ]:
from typing import List
from itertools import combinations, permutations, tee, groupby
from networkx.algorithms import shortest_path
from math import inf

def pairwise(iterable):
    # pairwise('ABCDEFG') --> AB BC CD DE EF FG
    a, b = tee(iterable)
    next(b, None)
    return zip(a, b)

def shortest_path_containing_nodes(graph: nx.Graph, nodes: List[int], source: int)-> List[int]:
  """
  Given a list of nodes to visit and a source node, find the shortest path that 
  contains all of those nodes.
  Calculates the pairwise shortest paths for each node in the list (and with the
   source), then iterates over combinations of those to find the shortest path 
   that contains all nodes.

  Note, this function runs in approximately `O(n!)` where n is the number of 
  nodes, which is obscenely slow for large `n`. Use this *only* for small values
  of `n`. Since the taxis in this problem have a limited (small) capacity, this 
  is a safe assumption. Use a different algorithm (and likely an approximation) 
  if you're implementing ride-sharing with buses. 

  :param graph: a networkx graph representing the road network
  :param nodes: a list of ints where each int is the index of a node in the 
    graph
  :param source: the index of the starting node as an int
  :returns: a list of node indices as ints
  """
  # Get all the nodes (add the source to the list, if it's not already in it)
  nodes_set = set(nodes)
  nodes_set.add(source)
  all_nodes = list(nodes_set)
  nodes_set.remove(source)
  destination_nodes = list(nodes_set)
  # Get all the pairs and find the shortest paths.
  pairs_of_points = combinations(all_nodes,2)
  shortest_pairs = {(x,y): shortest_path(graph, x,y) for x,y in pairs_of_points}
  # Add the reverse pairs to the dictionary, for sped-up look-up.
  for (x,y), path in shortest_pairs.copy().items():
    shortest_pairs[(y,x)] = list(reversed(path))
  # This speed-up assumes that "Distance between a pair of connected nodes is 
  # 1/20th of a mile" i.e. all edges have constant length
  shortest_pairs_lengths = {pair: len(path) for pair, path in shortest_pairs.items()}

  # print(shortest_pairs)
  # print(shortest_pairs_lengths)

  # Iterate over all possible orders of points 
  all_possible_paths = permutations(destination_nodes)
  # Source node must always be the first one. The car cannot teleport.
  all_possible_paths = [[source] + list(path) for path in all_possible_paths]
  
  # Iterate over all the paths and find the shortest one
  current_min_path_length = inf
  current_min_path = None
  for path in all_possible_paths: 
    # Calculate the length of the path by iterating over the path pairwise and 
    # adding up the lengths of shortest pairwise paths
    length = sum(shortest_pairs_lengths[(x,y)] for x,y in pairwise(path))
    sub_paths = [shortest_pairs[(x,y)] for x,y in pairwise(path)]
    full_path = [item for subpath in sub_paths for item in subpath]
    # print(full_path)
    if length < current_min_path_length:
      current_min_path_length = length
      current_min_path = full_path

  current_min_path = [x[0] for x in groupby(current_min_path)]

  # This is guaranteed to be the shortest path
  return current_min_path


In [ ]:
# Test shortest path containing nodes
# If these test functions run without throwing any errors, then the path contains all of the nodes.
# These functions do not test for the minimality of the algorithm, however, just if the path reaches all of its destinations

def test_nodes_source_in_min_path(graph: nx.Graph, nodes: List[int], source: int):
  for node in nodes:
    assert node in graph.nodes
  min_path = shortest_path_containing_nodes(graph, nodes, source)
  for node in nodes:
    assert node in min_path, f'[Error] Node {node} is not in the minimum path: {min_path}'
  assert source in min_path, f'[Error] Source {source} is not in the minimum path: {min_path}'
  return min_path

# 5 random test nodes
test_nodes = sample(test_roads.nodes, 5)

# Test min_path with first one as source and all 5 in nodes list:
source_in_list = test_nodes_source_in_min_path(test_roads, test_nodes, test_nodes[0])
# Test min_path with first one as source but not in nodes list:
source_not_in_list = test_nodes_source_in_min_path(test_roads, list(set(test_nodes) - {test_nodes[0]}), test_nodes[0])
# Make sure they are equivalent
assert source_in_list==source_not_in_list

print('Test nodes:', test_nodes)
print('Shortest path through nodes:', source_in_list)
print('Shortest path through nodes:', source_not_in_list)


Test nodes: [23, 190, 5, 28, 155]
Shortest path through nodes: [23, 161, 190, 66, 124, 86, 155, 86, 167, 160, 5, 64, 28]
Shortest path through nodes: [23, 161, 190, 66, 124, 86, 155, 86, 167, 160, 5, 64, 28]


In [ ]:
# Define a customer
import itertools

class Customer:
  counter = itertools.count()
  def __init__(self, pickup_location:int, dropoff_location:int):
    self.pickup_location = pickup_location
    self.drop_off_location = dropoff_location
    self.pickup_time_estimate = None
    self.id = next(Customer.counter)

  def __repr__(self):
    """
    Representation of customer.
    """
    return f'Customer({self.id}, {self.pickup_location}->{self.drop_off_location})'

In [ ]:
import itertools
import networkx as nx
from collections import defaultdict
# Define a car
class Car:
  """Contains the state and functionality necessary for updating a car in the simulation"""
  counter = itertools.count()

  def __init__(self, road_network:nx.Graph, initial_position:int, 
               capacity:int=5, max_speed:float=30):
    """
    :param road_network: NetworkX Graph representing the road network on which the car operates
    :param initial_position: integer representing the node index in `road_network` where the car starts
    :param capacity: integer representing the maximum number of passengers. Defaults to 5.
    :param max_speed: float representing the maximum distance (in miles) the car can drive in a single hour
    """
    self.id = next(Car.counter)
    self.road_network = road_network
    self.current_position = initial_position
    self.capacity = capacity
    self.max_speed = max_speed
    self.stopped = False
    self.total_distance = 0
    self.passengers = defaultdict(list) # (Key: drop_off node, Value: list(Customer) )
    self.passengers_count = 0
    self.queued_passengers = defaultdict(list) # (Key: pickup node, Value: list(Customer) )
    self.queued_count = 0

    # Route to take. List of node indices
    self.route: List[int] = []

  def board(self, customer: Customer, verbose=True) -> None:
    self.passengers[customer.drop_off_location].append(customer)
    self.passengers_count += 1
    self.queued_count -= 1
    if verbose:
      print(f'\tCar {self.id}: Picked up {customer} at location {self.current_position}')

  def pick_up_stop(self, start_node: int, verbose=True) -> None:
    """
    Pick up all customers who requested a pick-up at `start_node`.
    :param start_node: node index
    """
    if start_node in self.queued_passengers.keys():
      boarding = self.queued_passengers.pop(start_node)
      for customer in boarding: self.board(customer, verbose)
    
    self.stopped = False
  
  def deboard(self, passenger: Customer, verbose=True) -> None:
    if verbose:
      print(f'\tCar {self.id}: Dropped off {passenger} at location {self.current_position}')
    self.passengers_count -= 1

  def drop_off_stop(self, stop_node: int, verbose=True) -> None:
    """
    Drop off all passengers who requested a drop-off at `stop_node`.
    :param stop_node: node index 
    """
    if stop_node in self.passengers.keys():
      disboarding = self.passengers.pop(stop_node)
      for passenger in disboarding: self.deboard(passenger, verbose)
    
    self.stopped = False
  
  def __repr__(self):
    return f'Car(id={self.id}, Location={self.current_position}, Passengers={self.passengers}, Queued Passengers={self.queued_passengers}, Route={self.route})'

In [ ]:
# Define the simulation class
import numpy as np
from numpy.random import multinomial

class Simulation:
  def __init__(self, num_nodes: int = 200, avg_connectivity: int = 2, num_cars:int = 30, reservations_per_hour:int=100, minimum_drop_off_distance:int=20):
    Car.counter = itertools.count()
    Customer.counter = itertools.count()
    self.roads = generate_road_network(num_nodes, avg_connectivity)
    self.cars = [Car(self.roads, 0) for _ in range(num_cars)]
    self.current_time = 0
    self.distance_between_nodes = 1/20
    self.max_time = 8*60. # 8 hours
    self.reservations_per_hour = reservations_per_hour
    self.reservations = None
    self.customers = list()
    self.unassigned_customers = list()

    #Calculate the length of each pair of nodes in the graph
    self.length = dict(nx.all_pairs_shortest_path_length(self.roads))

    self.minimum_drop_off_distance = minimum_drop_off_distance

  def tick(self, verbose=True):
    """
    Move forward 1 minute.
    """
    for car in self.cars:
      self.move_car(car, verbose)
    self.current_time += 1

    # Create a list of 60 numbers that sum to a value between 100-150
    # This list corresponds to the number of reservations per tick
    # Creates a new list every 'hour'
    if self.current_time % 60 == 1:
      num_reservations = np.random.randint(self.reservations_per_hour, self.reservations_per_hour + 51)
      self.reservations = list(np.random.multinomial(num_reservations, np.ones(60)/60))

    tick_num_reservations = self.reservations[self.current_time % 60]
    if tick_num_reservations != 0: self.create_customers(tick_num_reservations)

  def move_car(self, car: Car, verbose=True) -> None:
    # The distance a car can move in one minute can be found by dimensional analysis
    # max_nodes = speed(mph) * [20 nodes/1mi] * [1 hr/60min] * [1min]
    # Assuming a speed of 30mph, this is 10 nodes/min.
    max_nodes = round(car.max_speed/ self.distance_between_nodes / 60)
    distance_traveled = 0
    # Make sure we can move to the next place on the route
    # Make sure there is a route and the next node is adjacent to the current
    if len(car.route):
      if car.current_position == car.route[0]:
        # If the first node in the route is our current position, just skip it.
        car.route.pop(0)
      if car.route[0] in self.roads[car.current_position]:  # Check adjacency
        while distance_traveled < max_nodes and not car.stopped and car.route:
          # Show we have moved the car by popping the front of the route and putting it into the current_position
          car.current_position = car.route.pop(0)
          distance_traveled += 1
          car.total_distance += 1
          if car.current_position in car.passengers.keys() or car.current_position in car.queued_passengers.keys():
            # We're at a stop, so load/unload the car next turn
            car.stopped = True
            break

        # We either moved as far as we can in one turn or reached a stop.
        # Unload passengers
        car.drop_off_stop(car.current_position, verbose)
        # Pick up customers at this stop if any
        car.pick_up_stop(car.current_position, verbose)
      else:
        # Recalculate route. Something went wrong
        self.recalculate_route(car)
        self.move_car(car)
    elif car.passengers_count + car.queued_count:
      # There is no route, but there are passengers either in the car or queued
      self.recalculate_route(car)
      # self.move_car(car)

  def recalculate_route(self, car: Car) -> None:
    """
    Recalculate the route of the given car.
    :param car: Car object representing the car whose route we recalculate
    """
    stops = list(car.passengers.keys()) + list(car.queued_passengers.keys())
    car.route = shortest_path_containing_nodes(self.roads, stops, car.current_position)

  def create_customers(self, number_of_customers):
    """
    Add customers to the simulation
    """
    for _ in range(number_of_customers):
      start_node = choice(list(self.roads.nodes))
      end_node = 0

      while True:
        candidate = choice(list(self.roads.nodes))
        if self.length[start_node][candidate] > self.minimum_drop_off_distance:
          end_node = candidate
          break
      
      customer = Customer(start_node, end_node)
      self.assign_car(customer)
      self.customers.append(customer)


  def assign_car(self, customer: Customer):
    """
    Assign the closest car to a customer
    """
    customer_pos = customer.pickup_location

    # All cars that have space for new customer
    eligble_cars = [car for car in self.cars if car.passengers_count + car.queued_count < car.capacity]
    
    if eligble_cars:
      # Find car whose distance from customer is lowest
      closest_car = min(eligble_cars, key=lambda car: self.length[customer_pos][car.current_position])

      # Assign customer to car 
      closest_car.queued_passengers[customer_pos].append(customer)
      closest_car.queued_count += 1

      # Calculate time estimate until pickup
      self.recalculate_route(closest_car)
      stops = len(closest_car.route) - 1
      customer.pickup_time_estimate = stops / round(closest_car.max_speed/ self.distance_between_nodes / 60)
    
    else: 
      print("no space")
      self.unassigned_customers.append(customer)
  
  def run_simulation(self, verbose=True):
    """
    Tick the simulation until the end of the day
    """
    while self.current_time < self.max_time:
      self.tick(verbose)
      if verbose:
        self.print_current()
  
  def print_current(self):
      print(f'Time: {self.current_time}/{self.max_time}, Total Distance Driven: {self.total_distance_traveled():.2f} mi')
      # print(self.cars)

  def total_distance_traveled(self):
    total = 0
    for car in self.cars: total += car.total_distance
    return total

In [ ]:
def run_multiple_simulations(num_trials:int, num_nodes=200, avg_connectivity=2, num_cars=30, reservations_per_hour=100, minimum_drop_off_distance=4):
  distances = {}
  for i in range(num_trials):
    verbose = not i
    print(f'Starting trial {i}')
    simulation=Simulation(num_nodes=num_nodes, avg_connectivity=avg_connectivity, num_cars=num_cars, reservations_per_hour=reservations_per_hour, minimum_drop_off_distance=minimum_drop_off_distance)
    # Draw Roads
    if verbose:
      nx.draw_kamada_kawai(simulation.roads)
    simulation.run_simulation(verbose=verbose)
    distances[i] = simulation.total_distance_traveled()
  print('----------------')
  for trial, distance in distances.items():
    print(f'Trial {trial} had total distance {distance}.')
  print(f'Average Distance traveled = {sum(distances.values())/len(distances)}')

In [ ]:
# Q4
run_multiple_simulations(25, num_nodes=200, avg_connectivity=2, num_cars=30, reservations_per_hour=100)

In [ ]:
#Q5
run_multiple_simulations(25, num_nodes=200, avg_connectivity=2, num_cars=60, reservations_per_hour=100)

In [ ]:
#Q6
run_multiple_simulations(25, num_nodes=200, avg_connectivity=4, num_cars=30, reservations_per_hour=100)

# Question 1
#### *1. Produce a pseudo code version of the algorithm needed to assign a vehicle to a customer.*
```
function assign_car(customer)
  cars_with_space = list()
  for each car in cars:
    if car.number_of_passengers + car.number_in_queue < max_capacity:
      cars_with_space.add(car)

  closest_car = from all the cars in cars_with_space:
                  find the one with the smallest distance to customer

  closest_car.customers_to_pick_up.add(customer)
```

# Question 2
#### *2. Produce a pseudo code version of the algorithm needed to plan a route for dropping off passengers in a given vehicle.*

```
Algorithm 2. Calculate the shortest path through a car's drop-off and pick-up nodes
  input: a car Car with passengers to drop off and pick up
  input: a connected graph Roads
  output: a list of nodes in the order of the shortest path 
    connecting all of the drop-off and pick-up nodes

  nodes_to_visit <- Car's drop-off and pick-up nodes
  Remove duplicates from nodes_to_visit.

  pairs_of_points <- Set of all pairs of nodes

  shortest_path_for_pairs <- {(x,y): shortest_path(Roads, x,y} for x,y in pairs_of_points}

  # NOTE: shortest_path between two nodes on a graph can be calculated 
  # fairly simply manually with Djistrka's algorithm or A*. 
  # Or even more simply, NetworkX has functions to quickly calculate 
  # shortest paths between pairs.

  shortest_pairs_lengths = {pair: len(path) for pair, path in shortest_pairs.items()}

  # The shortest path containing all points consists solely of the 
  # shortest paths between target nodes adjacent in the route.
  # Proof: If the path between two adjacent target nodes in the route 
  # did not take the shortest subpath, then changing that subroute 
  # with the shortest subpath between those two nodes would result
  # in a shorter overall path.

  # This means we reduce the search space to all possible permutations 
  # of destination nodes
  all_possible_paths = permutations(destination_nodes)
  
  # Iterate over all the paths and find the shortest one
  current_min_path_length = inf
  current_min_path = None
  for path in all_possible_paths: 
    # Calculate the length of the path by iterating over the path pairwise and 
    # adding up the lengths of shortest pairwise paths
    length = sum(shortest_pairs_lengths[(x,y)] for x,y in pairwise(path))
    sub_paths = [shortest_pairs[(x,y)] for x,y in pairwise(path)]
    full_path = [item for subpath in sub_paths for item in subpath]
    if length < current_min_path_length:
      current_min_path_length = length
      current_min_path = full_path

  return current_min_path
```